# Aplicativo Web de Análise de Sentimentos

Este notebook se baseia no [github do Deep Learning Nanodegree da Udacity](https://github.com/udacity/deep-learning-v2-pytorch).

---

Neste notebook, iremos utilizar o serviço da Amazon SageMaker para construir um modelo de predição de sentimentos de reviews de filmes. Iremos realizar o deploy deste modelo e criaremos uma aplicação web bem simples para que possamos interagir.

## Visão Geral

Geralmente, ao utilizar um notebook no SageMaker, iremos percorrer os passos abaixo. Claro que nem todos eles serão necessários em todos os projetos e, ainda, dependendo da aplicação, alguns passos terão que ser feitos de forma diferente. O formato geral, entretanto, será apresentado aqui.

0. Preliminares.
1. Obtenção dos dados.
2. Processamento/preparação dos dados.
3. Upload dos dados processados para o S3.
4. Treinamento do modelo.
5. Teste do modelo treinado (geralmente, utilizando um job de batch transform).
6. Deploy do modelo treinado.
7. Utilização do modelo em deploy.

## Passo 0: Preliminares

* Crie uma conta AWS [aqui](https://aws.amazon.com/premiumsupport/knowledge-center/create-and-activate-aws-account/)
* Entre no [console da AWS](https://aws.amazon.com/console/).

<img src="images/01.png">

* Não confunda SageMaker com EC2. Estaremos utilizando o SageMaker. Iremos precisar das duas seguintes instâncias: `ml.m4.xlarge` e `ml.p2.xlarge`. Caso você não tenha acesso quando formos precisar (provavelmente você só tenha acesso ao primeiro), você deverá abrir um chamado no [suporte da AWS](https://console.aws.amazon.com/support/home) requisitando a liberação de uma máquina do tipo desejado.

    1. Crie um novo caso

    <img src="images/02.png">

    2. Vá em _Service Limit Increase_

    <img src="images/03.png">

    3. Em _Case details_, selecione "SageMaker" no campo _Limit type_.

    <img src="images/04.png">

    4. Logo abaixo, haverá uma seção de _Requests_. Você deve preenchê-la de acordo com as informações que você precisa.

    <img src="images/05.png">

    5. Logo abaixo, preencha o campo de _Case description_. Basta informar que você gostaria de treinar modelos de deep learning utilizando o Amazon SageMaker.

    6. Preencha suas informações de contato e clique em _Submit_.

    7. O processo para liberação de uma nova instância não é instantâneo, podendo demorar de algumas horas até alguns dias.

* Agora, iremos configurar nossa instância de notebook no SageMaker.

    >**Nota: assim que o notebook é criado, ele estará com o estado _InService_, ou seja, rodando. Tenha em mente que você será cobrado de acordo com o tempo em que o notebook estiver rodando. Sendo assim, quando não for mais utilizá-lo, lembre-se de colocá-lo em _Stop_.**

    1. Procure por _Amazon SageMaker_.
    
    <img src="images/06.png">
    
    2. Clique em _Instâncias do bloco de anotações_ e, depois, em _Criar instância do bloco de anotações_. Você pode escolher o nome que quiser para o seu notebook. Utilize um `ml.t2.medium`, que será suficiente por enquanto e é gratuito.
    
    <img src="images/07.png">
    
    3. Logo abaixo, em _Função do IAM_, selecione _Criar uma nova função_. A única alteração que precisa ser feita é selecionar _Nenhum_ em _Buckets do S3 especificados_.
    
    <img src="images/08.png">
    
    4. Por fim, clique em _Criar instância do bloco de anotações_ e aguarde até que a instância esteja disponível.
    
    5. Para colocar o notebook como _Stop_ ou _InService_, apenas clique no nome do notebook e clique na opção desejada. Para abrir o notebook, clique em _Abrir o Jupyter_.
    
* Por fim, devemos colocar o notebook dentro da instância para que possamos executá-lo por lá.

    1. Iremos clonar o notebook a partir de um repositório no github. Para isso, clique no menu dropdown _new_ e selecione _Terminal_.
    <pre><code>
    cd SageMaker
    git clone https://github.com/mendelson/Tutorial-AWS-SageMaker-Deploy.git
    exit
    </code></pre>

    2. Feche a janela do terminal e veja que agora seu notebook já está lá. Agora o seu ambiente está preparado!

## Passo 1: Obtenção dos dados

O dataset que iremos utilizar é bem popular entre pesquisadores de NLP: [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/). Ele é composto por reviews de filmes postadas no site [imdb.com](http://www.imdb.com/), sendo cada uma marcada como '**pos**itiva', se o escritor gostou do filme, ou '**neg**ativa' , caso contrário.

Vamos usar um pouco de magic para fazer o download e extração do dataset.

In [ ]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

## Passo 2: Processamento/preparação dos dados

Os dados que baixamos está dividido em vários arquivos, cada um contendo uma única review. Será muito mais fácil combinar esses arquivos em apenas dois grandes arquivos: um para treinamento e outro para teste.

In [ ]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [ ]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

In [ ]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [ ]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

In [ ]:
train_X[100]

## Processando os dados

Agora que já temos os conjuntos de treinamento e de teste, precisamos processar os dados originais em algo legível pelo nosso algoritmo de machine learning. Primeiro, iremos remover as formatações HTML e quaisquer caracteres não alpha numéricos. Iremos utilizar, para isso, o módulo de expressões regulares do Python.

In [ ]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def review_to_words(review):
    words = REPLACE_NO_SPACE.sub("", review.lower())
    words = REPLACE_WITH_SPACE.sub(" ", words)
    return words

In [ ]:
review_to_words(train_X[100])

In [ ]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_web_app")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [ ]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

### Extração das features de Bag-of-Words

Para o modelo que iremos implementar, iremos transformar cada review em uma representação de features de Bag-of-Words. Note que utilizaremos apenas o conjunto de treinamento para construir essa representação. Se você quiser saber mais sobre como o método Bag-of-Words funciona, consulte [este](https://machinelearningmastery.com/gentle-introduction-bag-words-model/) tutorial introdutório.

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib
# joblib is an enhanced version of pickle that is more efficient for storing NumPy arrays

def extract_BoW_features(words_train, words_test, vocabulary_size=5000,
                         cache_dir=cache_dir, cache_file="bow_features.pkl"):
    """Extract Bag-of-Words for a given set of documents, already preprocessed into words."""
    
    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = joblib.load(f)
            print("Read features from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Fit a vectorizer to training documents and use it to transform them
        # NOTE: Training documents have already been preprocessed and tokenized into words;
        #       pass in dummy functions to skip those steps, e.g. preprocessor=lambda x: x
        vectorizer = CountVectorizer(max_features=vocabulary_size)
        features_train = vectorizer.fit_transform(words_train).toarray()

        # Apply the same vectorizer to transform the test documents (ignore unknown words)
        features_test = vectorizer.transform(words_test).toarray()
        
        # NOTE: Remember to convert the features using .toarray() for a compact representation
        
        # Write to cache file for future runs (store vocabulary as well)
        if cache_file is not None:
            vocabulary = vectorizer.vocabulary_
            cache_data = dict(features_train=features_train, features_test=features_test,
                             vocabulary=vocabulary)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                joblib.dump(cache_data, f)
            print("Wrote features to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        features_train, features_test, vocabulary = (cache_data['features_train'],
                cache_data['features_test'], cache_data['vocabulary'])
    
    # Return both the extracted features as well as the vocabulary
    return features_train, features_test, vocabulary

In [ ]:
# Extract Bag of Words features for both training and test datasets
train_X, test_X, vocabulary = extract_BoW_features(train_X, test_X)

In [ ]:
len(train_X[100])

## Passo 3: Upload dos dados processados para o S3

Agora que criamos a representação das features para os conjuntos de treinamento e de teste, é hora de começarmos a trabalhar com o classificador XGBoost do SageMaker.

### Escrevendo o dataset

O classificador XGBoost que iremos utilizar requer que o dataset esteja armazenado no Amazon S3. Para isso, iremos separar o conjunto de treinamento em duas partes: os dados para treinar o modelo e um conjunto de validação. Então, iremos armazenar esses dados em um arquivo local e, depois, colocá-lo no S3.

O conjunto de teste, por sua vez, também será salvo em arquivo e colocado no S3. Faremos esses passos para podermos utilizar a funcionalidade de Batch Transform do SageMaker para testar o nosso modelo após o treinamento.

In [ ]:
import pandas as pd

# Earlier we shuffled the training dataset so to make things simple we can just assign
# the first 10 000 reviews to the validation set and use the remaining reviews for training.
val_X = pd.DataFrame(train_X[:10000])
train_X = pd.DataFrame(train_X[10000:])

val_y = pd.DataFrame(train_y[:10000])
train_y = pd.DataFrame(train_y[10000:])

A documentação do algoritmo do XGBoost do SageMaker requer que os conjuntos de treinamento e validação não contenham headers nem índices, e que a label apareça primeiro para cada amostra.

Para mais informações sobre este e outros algoritmos, a documentação para desenvolvedores do SageMaker pode ser encontrada [aqui](https://docs.aws.amazon.com/sagemaker/latest/dg/).

In [ ]:
# First we make sure that the local directory in which we'd like to store the training and validation csv files exists.
data_dir = '../data/sentiment_web_app'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [ ]:
pd.DataFrame(test_X).to_csv(os.path.join(data_dir, 'test.csv'), header=False, index=False)

pd.concat([val_y, val_X], axis=1).to_csv(os.path.join(data_dir, 'validation.csv'), header=False, index=False)
pd.concat([train_y, train_X], axis=1).to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

In [ ]:
# To save a bit of memory we can set text_X, train_X, val_X, train_y and val_y to None.

test_X = train_X = val_X = train_y = val_y = None

### Fazendo o upload dos arquivos de treinamento e validação para o S3

O serviço S3 da Amazon nos permite armazenar arquivos que podem ser acessados pelos modelos internos (como o XGBoost) e também por modelos customizados (como PyTorch).

Há duas abordagens para lidar com o S3: uma baixa nível e outra alto nível. Por simplicidade, iremos utilizar a abordagem alto nível, que nos permitirá evoluir mais rapidamente.

O método `upload_data()` pertence ao objeto que representa a sessão atual do SageMaker. O que esse método faz é subir os dados para o bucket padrão (que é automaticamente criado se ainda não existir) dentro do caminho descrito pela variável `key_prefix`.

Para mais informações, consulte a __[documentação da API do SageMaker](http://sagemaker.readthedocs.io/en/latest/)__ e o __[Guia do Desenvolvedor do SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/)__.

In [ ]:
import sagemaker

session = sagemaker.Session() # Store the current SageMaker session

# S3 prefix (which folder will we use)
prefix = 'sentiment-web-app'

test_location = session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
val_location = session.upload_data(os.path.join(data_dir, 'validation.csv'), key_prefix=prefix)
train_location = session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

## Passo 4: Criando o modelo XGBoost

Agora que os dados já foram colocados em seu devido lugar, é hora de criar o modelo XGBoost.

No SageMaker, um modelo é composto por 3 diferentes objetos no ecosistema do SageMaker, que interagem entre si:

- Artefatos do Modelo
- Código de Treinamento (Container)
- Código de Inferência (Container)

Os artefatos do modelo são o que podem ser considerados como sendo o próprio modelo. Por exemplo, se você estiver construindo uma rede neural, os artefatos do modelo seriam os pesos das várias camadas. No nosso caso, para o modelo XGBoost, os artefatos são as próprias árvores que serão criadas durante o treinamento.

Os outros dois objetos são utilizados para manipular os artefatos de treinamento. Mais precisamente, o código de treinamento utiliza os dados de treinamento e cria os artefatos do modelo, enquanto que o código de inferência utiliza os artefatos do modelo para realizar predições sobre novos dados.

O SageMaker executar os códigos de treinamento e inferência por meio de containers Docker. Pense em um container como sendo uma forma de empacotar o código de tal forma que as dependências não venham a ser um problema.

In [ ]:
from sagemaker import get_execution_role

# Our current execution role is required when creating the model as the training
# and inference code will need to access the model artifacts.
role = get_execution_role()

In [ ]:
# We need to retrieve the location of the container which is provided by Amazon for using XGBoost.
# As a matter of convenience, the training and inference code both use the same container.
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(session.boto_region_name, 'xgboost')

In [ ]:
# First we create a SageMaker estimator object for our model.
xgb = sagemaker.estimator.Estimator(container, # The location of the container we wish to use
                                    role,                                    # What is our current IAM Role
                                    train_instance_count=1,                  # How many compute instances
                                    train_instance_type='ml.m4.xlarge',      # What kind of compute instances
                                    output_path='s3://{}/{}/output'.format(session.default_bucket(), prefix),
                                    sagemaker_session=session)

# And then set the algorithm specific parameters.
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        early_stopping_rounds=10,
                        num_round=500)

### Treinamento do modelo XGBoost

Agora que o nosso modelo já foi configurado, precisamos apenas conectar os conjuntos de treinamento e validação e, então, o SageMaker realizará os cálculos.

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data=val_location, content_type='csv')

In [ ]:
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

## Passo 5: Teste do modelo treinado

Agora que já treinamos o nosso modelo XGBoost, é hora de testar sua performance. Para isso, iremos utilizar a funcionalidade de Batch Transform do SageMaker.

Batch Transform é uma forma conveniente de realizar inferência em um grande dataset sem realizar cálculos realtime. Ou seja, não precisamos necessariamente utilizar os resultados do nosso modelo imediatamente e, em vez disso, podemos realizar inferência em uma grande quantidade de amostras.

Um exemplo disso seria a elaboração de um relatório empresarial apenas no fim do mês.

Para realizar essa tarefa, precisamos primeiro criar um objeto transformador a partir do nosso modelo.

In [ ]:
xgb_transformer = xgb.transformer(instance_count = 1, instance_type = 'ml.m4.xlarge')

Agora, realizaremos a transformação. Precisamos especificar corretamente o tipo dos dados que estamos mandando, para que possam ser serializados corretamente. No nosso caso, o modelo receberá dados csv, então especificamos `text/csv`. Além disso, caso o conjunto de teste seja muito grande para processar de uma só vez, precisamos especificar como o arquivo deve ser cortado. Já que cada linha forma uma única entrada, diremos ao SageMaker que os dados podem ser quebrados em cada linha.

In [ ]:
xgb_transformer.transform(test_location, content_type='text/csv', split_type='Line')

O job de transformação está rodando, mas ele realiza sua tarefa em background. Já que queremos esperar até que o job de transformação seja completado, chamaremos o método `wait()`.

In [ ]:
xgb_transformer.wait()

Agora sim, o job de transformação foi executado e as predições de cada review foram salvas no S3. Vamos copiar o arquivo para o diretório `data_dir`.

In [ ]:
!aws s3 cp --recursive $xgb_transformer.output_path $data_dir

A última etapa consiste em ler a saída do nosso modelo, convertê-la em algo legível (`1` para positivo e `0` para negativo) e compará-la com as labels reais.

In [ ]:
predictions = pd.read_csv(os.path.join(data_dir, 'test.csv.out'), header=None)
predictions = [round(num) for num in predictions.squeeze().values]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

## Passo 6: Deploy do modelo treinado

Uma vez que o modelo está construído e treinado, o SageMaker armazena os artefatos do modelo e podemos utilizá-los para realizar o deploy de um endpoint (código de inferência).

Realizar o deploy de um endpoint é como treinar um modelo, com algumas diferenças importantes. A primeira é que um modelo em produção não altera os artefatos do modelo. Outra diferença é que o modelo fica rodando até que o façamos parar explicitamente. Se não o pararmos, ele continuará rodando e seremos cobrados por todo este tempo.

**Se você não estiver mais utilizando o endpoint, desligue-o!**

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

### Testando o modelo (de novo)

Agora que o endpoint está em deploy, podemos enviar os dados de teste para ele e receber os resultados da inferência. Esse é o mesmo processo que fizemos anteriormente com o Batch Transform, porém agora iremos realizar as inferências em um endpoint em funcionamento para ver se está tudo certo.

Ao utilizar o endpoint, é importante termos em mente que estamos limitados na quantidade de informação que podemos enviar em cada chamada, então precisamos quebrar o conjunto de teste em chunks e enviar cada chunk. Também precisamos serializar nossos dados antes de enviá-los para o endpoint para termos certeza de que os dados foram transmitidos corretamente. Felizmente, o SageMaker realiza a serialização para nós.

In [ ]:
from sagemaker.predictor import csv_serializer

# We need to tell the endpoint what format the data we are sending is in so that SageMaker can perform the serialization.
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer

In [ ]:
# We split the data into chunks and send each chunk seperately, accumulating the results.

def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])
    
    return np.fromstring(predictions[1:], sep=',')

In [ ]:
test_X = pd.read_csv(os.path.join(data_dir, 'test.csv'), header=None).values

predictions = predict(test_X)
predictions = [round(num) for num in predictions]

Finalmente, vamos ver a acurácia do nosso modelo.

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

Os resultados aqui deveriam ser os mesmos que obtivemos anteriormente com o Batch Transform.

### Limpando o ambiente

Agora que sabemos que o deploy funcionou corretamente, iremos desligá-lo. Lembre-se que quanto mais tempo o endpoint rodar, mais seremos cobrados. Já que ainda vamos realizar mais algumas operações antes de utilizar o modelo, iremos desligá-lo para evitar gastos.

In [ ]:
xgb_predictor.delete_endpoint()

## Passo 7: Colocando o modelo para trabalhar!

Nosso objetivo é realizar o deploy do nosso modelo e depois acessá-lo utilizando uma aplicação web bem simples. Queremos utilizar essa aplicação para enviar os dados fornecidos pelo usuário (uma review) para o endpoint (o modelo) e exibir o resultado.

Atualmente, a única forma de enviar dados para o endpoint é utilizando a API do SageMaker. O endpoint criado pelo SageMaker requer que a entidade que esteja o acessando tenha as permissões corretas, então seria necessário realizar uma autenticação da nossa aplicação web com a AWS. Isso nos leva a uma série de dificuldades. Isso é algo muito mais elaborado do que o que queremos fazer aqui. Sendo assim, adotaremos uma abordagem alternativa.

Iremos criar um novo endpoint que não requer autenticação e que age como um proxy para o endpoint do SageMaker.

Também evitaremos realizar qualquer processamento de dados na própria aplicação web. Faremos tudo no backend, utilizando o serviço Lambda da Amazon.

<img src="images/Web App Diagram.svg">

O diagrama acima mostra como os serviços irão interagir entre sim. À direita está o modelo que treinamos; à esquerda, a aplicação web que irá interagir com o usuário.

No meio, iremos construir uma função Lambda, que é uma função escrita em Python que pode ser executada quando um evento específico ocorrer. Essa função irá processar os dados recebidos. Também daremos à essa função permissão para enviar e receber dados do endpoint do SageMaker.

Por fim, o método que iremos utilizar para executar a função Lambda é um novo endpoint que iremos criar utilizando Gateway API. Esse endpoint será uma URL que fica à espera por dados enviados para ela. Quando ela os recebe, os dados são passados para a função Lambda e então retorna os dados que a função Lambda retorna. Essencialmente, o Gateway irá agir como uma interface que permite que a nossa aplicação web se comunique com a função Lambda.

### Processando uma única review

Por agora, suponha que recebemos uma única review de um usuário em forma de string, conforme abaixo:

In [ ]:
test_review = "Nothing but a disgusting materialistic pageant of glistening abed remote control greed zombies, totally devoid of any heart or heat. A romantic comedy that has zero romantic chemestry and zero laughs!"

In [ ]:
test_words = review_to_words(test_review)
print(test_words)

Agora, iremos construir o encoding de BoW.

In [ ]:
def bow_encoding(words, vocabulary):
    bow = [0] * len(vocabulary) # Start by setting the count for each word in the vocabulary to zero.
    for word in words.split():  # For each word in the string
        if word in vocabulary:  # If the word is one that occurs in the vocabulary, increase its count.
            bow[vocabulary[word]] += 1
    return bow

In [ ]:
test_bow = bow_encoding(test_words, vocabulary)
print(test_bow)

In [ ]:
len(test_bow)

Agora que sabemos como construir o encoding BoW, precisamos subir novamente o endpoint para podermos enviar dados para ele.

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

Funções Python que são utilizadas em funções Lambda têm acesso à mais uma biblioteca da Amazon, chamada `boto3`. Essa biblioteca fornece uma API para trabalhar com os serviços da Amazon, inclusive o SageMaker.

Vamos pegar um handle para a runtime do SageMaker.

In [ ]:
import boto3

runtime = boto3.Session().client('sagemaker-runtime')

Agora que temos acesso ao runtime do SageMaker, iremos invocar um endpoint que já foi criado. Entretanto, precisamos fornecer ao SageMaker o nome do endpoint em deploy. Para descobrir isso, podemos imprimir utilizando o objeto `xgb_predictor`.

In [ ]:
xgb_predictor.endpoint

Utilizando o runtime do SageMaker e o nome do nosso endpoint, podemos invocar o endpoint e enviar os dados de `test_bow`.

In [ ]:
response = runtime.invoke_endpoint(EndpointName = xgb_predictor.endpoint, # The name of the endpoint we created
                                       ContentType = 'text/csv',                     # The data format that is expected
                                       Body = test_bow)

Por que recebemos um erro?

Porque tentamos enviar para o endpoint uma lista de inteiros, mas ele espera receber dados do tipo `text/csv`.

Precisamos realizar essa conversão.

In [ ]:
response = runtime.invoke_endpoint(EndpointName = xgb_predictor.endpoint, # The name of the endpoint we created
                                       ContentType = 'text/csv',                     # The data format that is expected
                                       Body = ','.join([str(val) for val in test_bow]).encode('utf-8'))

In [ ]:
print(response)

A resposta do nosso modelo é um dict um tanto quando complicado com um monte de informações. Estamos interessados no campo `'Body'`.

In [ ]:
response = response['Body'].read().decode('utf-8')
print(response)

Agora que sabemos como processar os dados de entrada, podemos montar a infraestrutura para subir a nossa aplicação web. Para isso, iremos utilizar os serviços da Amazon Lambda e API Gateway.

Lambda é um serviço que permite que alguém escreva um código relativamente simples e que ele seja executado quando um trigger ocorrer.

API Gateway é uma serviço que permite a criação de endpoints HTTP (URL) que são conectados a outro serviço AWS. Um de seus benefícios é que você pode decide quais credenciais, se alguma, são necessárias para acessar esses endpoints.

No nosso caso, iremos setar um endpoint HTTP através da API Gateway que será aberto ao público. Sendo assim, sempre que alguém enviar dados para o nosso endpoint público, a função Lambda será executada, que irá enviar a review para o endpoint do nosso modelo e, então, irá retornar o resultado.

### Configurando a função Lambda

A primeira coisa a ser feita é a configuração da função Lambda.

#### Parte A: Crie uma Função IAM para a função Lambda

Já que queremos que a função Lambda chame o endpoint do SageMaker, nós precisamos ter certeza que temos as devidas permissões. Para isso, iremos construir uma Função que iremos fornecer à função Lambda.

Utilizando o Console AWS, navegue até a página **IAM** e clique em **Funções**. Clique em **Criar função**. Certifique-se de que **Serviço da AWS** esteja selecionado e escolha **Lambda** como o serviço que irá utilizar esta Função. Então, clique em **Próximo: Permissões**.

Na caixa de pesquisa, digite `sagemaker` e marque a caixa de seleção referente a **AmazonSageMakerFullAccess**. Então, clique em **Próximo: Tags**.

Nesta tela, não adicione nada. Apenas clique em **Próximo: Revisar**.

Dê um nome para essa Função. Use um nome que você irá se lembrar, por exemplo `LambdaAulaDeployRole`. Finalmente, clique em **Criar função**.

#### Parte B: Crie uma função Lambda

Agora, iremos criar a própria função Lambda. Antes, precisamos de duas informações:

- O nome do endpoint e
- O objeto de vocabulário.

Iremos copiar essas informações para a nossa função Lambda após criá-la.

No console AWS, navegue até Lambda e clique em **Criar função**. Certifique-se que **Criar do zero** esteja selecionado. Dê um nome para a sua função de forma que você se lembre desse nome, tal como `LambdaAulaDeploy`. Certifique-se também de que a runtime **Python 3.8** esteja selecionada. Abra as opções de **Escolher ou criar uma função de execução**, selecione **Usar uma função existente** e selecione a Função IAM que criamos anteriormente. Por fim, clique em **Criar função**.

Na tela seguinte, você verá várias informações. Desça a tela e você verá um campo para preenchimento de código fonte em Python. É aí onde escreveremos a nossa função Lambda.

Copie e cole o código abaixo neste campo.

```python
# We need to use the low-level library to interact with SageMaker since the SageMaker API
# is not available natively through Lambda.
import boto3

# And we need the regular expression library to do some of the data processing
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def review_to_words(review):
    words = REPLACE_NO_SPACE.sub("", review.lower())
    words = REPLACE_WITH_SPACE.sub(" ", words)
    return words
    
def bow_encoding(words, vocabulary):
    bow = [0] * len(vocabulary) # Start by setting the count for each word in the vocabulary to zero.
    for word in words.split():  # For each word in the string
        if word in vocabulary:  # If the word is one that occurs in the vocabulary, increase its count.
            bow[vocabulary[word]] += 1
    return bow


def lambda_handler(event, context):
    
    vocab = "*** ACTUAL VOCABULARY GOES HERE ***"
    
    words = review_to_words(event['body'])
    bow = bow_encoding(words, vocab)

    # The SageMaker runtime is what allows us to invoke the endpoint that we've created.
    runtime = boto3.Session().client('sagemaker-runtime')

    # Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given
    response = runtime.invoke_endpoint(EndpointName = '***ENDPOINT NAME HERE***',# The name of the endpoint we created
                                       ContentType = 'text/csv',                 # The data format that is expected
                                       Body = ','.join([str(val) for val in bow]).encode('utf-8')) # The actual review

    # The response is an HTTP response whose body contains the result of our inference
    result = response['Body'].read().decode('utf-8')
    
    # Round the result so that our web app only gets '1' or '0' as a response.
    result = round(float(result))

    return {
        'statusCode' : 200,
        'headers' : { 'Content-Type' : 'text/plain', 'Access-Control-Allow-Origin' : '*' },
        'body' : str(result)
    }
```

Substitua `**ENDPOINT NAME HERE**` pelo nome do endpoint que realizamos o deploy anteriormente. Você pode ver esse nome executando a célula abaixo.

In [ ]:
xgb_predictor.endpoint

Também precisaremos copiar o dicionário de vocabulário no campo apropriado no código acima, no lugar de `*** ACTUAL VOCABULARY GOES HERE ***`. A célula abaixo exibe o dicionário de vocabulário de forma fácil de copiar e colar.

In [ ]:
print(str(vocabulary))

Após realizar esses passos, clique em **Salvar**. Sua função Lambda está pronta e já executando. Agora precisamos criar uma forma de a nossa aplicação web executar a função Lambda.

### Configurando API Gateway

Com a função Lambda pronta, iremos criar uma nova API utilizando API Gateway, que irá acionar a função Lambda que acabamos de criar.

No console AWS, navegue até **API Gateway**. Selecione **Compilar** em **API REST**.

Na tela seguinte, selecione **API nova** e dê um nome para a API, por exemplo, `APIAulaDeploy`. Então clique em **Criar API**.

Já temos a API pronta, porém ela ainda não faz nada. Queremos que ela dê um trigger na função Lambda.

Selecione o menu dropdown **Ações** e clique em **Criar método**. Um novo método em branco será criado. Selecione seu menu dropdown, selecione **POST* e clique no sinal de check ao seu lado.

No ponto de integração, selecione **Função Lambda** e clique em **Usar a integração de proxy do Lambda**. Essa opção garante que os dados enviados para a API será enviada diretamente para a função Lambda sem processamento. Ela também garante que o valor de retorno será um objeto apropriado, já que não será processado pela API Gateway.

No campo **Função Lambda**, digite o nome da função Lambda que criamos anteriormente. Clique em **Salvar**. Clique em **OK** caso apareça uma pop-up, dando permissão para a API Gateway invocar a função Lambda.

Por fim, selecione novamente o menu dropdown **Ações** e clique em **Implantar API**. Você irá criar um novo estágio de deploy e dê a ele o nome que quiser, como `prod`.

Agora nós setamos uma API pública para acessar o nosso modelo do SageMaker. Certifique-se de copiar a URL fornecida para invocá-la. Essa URL pode ser encontrada na parte superior da tela, marcada em azul próxima ao texto **Invocar URL**.

## Passo 7: Deploy da aplicação web

Note que há um arquivo HTML na pasta deste nosso projeto (`index.html`).

Abra este arquivo em um editor de texto e substitua **\*\*REPLACE WITH PUBLIC API URL\*\*** pela URL que você acabou de copiar.

Agora, abra o `index.html` em um navegador web, digite uma review em inglês e a envie.

Você também pode colocar esse arquivo HTML em algum site de sua preferência, tal como um github, e compartilhar o link para seus colegas testarem seu sistema.

> **Nota**: mais uma vez, lembre-se de desligar tudo quando não for mais utilizar o seu sistema. Caso contrário, você continuará sendo cobrado pelo uso!

### Delete o endpoint

Lembre-se de sempre desligar o endpoint quando ele não for mais necessário. Você será cobrado de acordo com o tempo que o endpoint rodar, então não se esqueça de desligá-lo.

In [ ]:
xgb_predictor.delete_endpoint()

## Opcional: limpando a casa

A instância do notebook do SageMaker, por padrão, possui pouco espaço de disco. Se você precisar e quiser, execute a célula abaixo para limpar o seu espaço.

In [ ]:
# First we will remove all of the files contained in the data_dir directory
!rm $data_dir/*

# And then we delete the directory itself
!rmdir $data_dir

# Similarly we remove the files in the cache_dir directory and the directory itself
!rm $cache_dir/*
!rmdir $cache_dir

# Sugestão de Projeto Final

Como uma opção de projeto final, realize o deploy do modelo de style transfer e crie uma aplicação web amigável para que possamos enviar uma imagem de conteúdo, uma imagem de estilo e que nos retorne a imagem estilizada.

O seguinte notebook, na pasta `Project`, será de grande ajuda para você: https://github.com/mendelson/sentiment-analysis-deploy-AWS.